In [5]:
import FinanceDataReader as fdr
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pmdarima.arima import ndiffs
import pmdarima as pm



krx_1 = pd.read_csv('./KS200.csv')

krx_1

C:\python38-64\lib\site-packages\pmdarima\__init__.py:107: UserWarning: pmdarima support for 32-bit systems is ending with Python 3.10.Your system is not guaranteed to work going forward
  _warnings.warn(


,Date,KS200
0,2016-03-02,238.63
1,2016-03-03,240.09
2,2016-03-04,239.74
3,2016-03-07,240.21
4,2016-03-08,238.44
...,...,...
1491,2022-03-25,362.06
1492,2022-03-28,362.06
1493,2022-03-29,363.77
1494,2022-03-30,364.50


In [ ]:
y_train = krx_1['KS200'][:int(0.7*len(krx_1))]
y_test = krx_1['KS200'][int(0.7*len(krx_1)):]
y_train.plot()
y_test.plot()

In [ ]:
kpss_diffs = ndiffs(y_train, alpha=0.05, test='kpss', max_d=6)
adf_diffs = ndiffs(y_train, alpha=0.05, test='adf', max_d=6)
n_diffs = max(adf_diffs, kpss_diffs)

print(f"추정된 차수 d = {n_diffs}")

In [ ]:
model = pm.auto_arima(y = y_train        # 데이터
                      , d = 1            # 차분 차수, ndiffs 결과!
                      , start_p = 0 
                      , max_p = 5   
                      , start_q = 0 
                      , max_q = 5   
                      , m = 1       
                      , seasonal = False # 계절성 ARIMA가 아니라면 필수!
                      , stepwise = True
                      , trace=True
                      )

In [ ]:
model = pm.auto_arima (y_train, d = 1, seasonal = False, trace = True)
model.fit(y_train)

 1. 잔차가 백색잡음 과정인지 (=정상성을 만족하는지), 정규성 및 등분산성을 만족하는지 파악합니다.
   * Ljung-Box (Q) / Heteroskedasticity (H) / Jarque-Bera (JB) 검정 만족 여부를 파악하실 수 있습니다.

 2. Ljung-Box (Q) 융-박스 검정 통계량는 잔차가 백색잡음인지 검정한 통계량입니다. 0.5가 적절함

 3. Prob (Q) 값을 보면 0.65이므로 유의수준 0.05에서 귀무가설을 기각하지 못합니다. 
 
 4. Ljung-Box (Q) 통계량의 귀무가설은 “잔차(residual)가 백색잡음(white noise) 시계열을 따른다”이므로, 
   * 위 결과를 통해 시계열 모형이 잘 적합되었고 남은 잔차는 더이상 자기상관을 가지지 않는 백색 잡음임을 확인할 수 있습니다.

 5. Jarque-Bera (JB) 자크-베라 검정 통계량은 잔차가 정규성을 띠는지 검정한 통계량입니다.
    * 쟈크베라의 검정(Jarque-Bera Test)은 적합도(Goodness-of-ift) 검정이다. 
    * 쟈크베라의 검정(Jarque-Bera Test)은 왜도(Skewness)와 첨도(Kurtosis)가 정규분포(normal distribution)로 보기에 적합한지에 대한 적합도(Goodness-of-fit) 검정에 사용한다.
    * 쟈크베라의 검정(Jarque-Bera Test)에 결과값은 언제나 양의 숫자다. 그리고 결괏값이 0에서 멀리 떨어진 값이 나오면, 그때는 정규분포(normal distiribuiton)에 적합하지 않다고 판정한다. 
    * 왜도 값이 -2~2 정도의 치우침은 왜도가 크지 않다고 판단합니다.
    * 왜도 skewness : 분포가 정규분포에 비해서 얼마나 비대칭인지 나타내는 척도
    * 절대값 3 미만이면 기준에 부합합니다.

    * 첨도 kurtosis : 샘플의 점수가 평균을 중심으로 가까이 몰려 있을수록 분포의 정점은 더욱 뾰족한 모양, 즉 첨도가 커진다. 
    * 첨도가 크면 이상치(outlier)가 많습니다.
    * 왜도 절대값 3미만, 첨도 절대값 7미만 이면 기준에 부합되는 것으로 봅니다.


 * Prob(JB)값을 보면 0.00으로 유의 수준 0.05에서 귀무가설을 기각합니다. 
 * Jarque-Bera (JB) 통계량의 귀무가설은 “잔차가 정규성을 만족한다”이므로, 
 * 위 결과를 통해 “잔차가 정규성을 따르지 않음”을 확인할 수 있습니다.


In [ ]:

print(model.summary())



In [ ]:
model.plot_diagnostics(figsize=(16, 8))
plt.show()

In [ ]:
# 테스트 데이터 개수만큼 예측
y_predict = model.predict(n_periods=len(y_test) + 10) 
#y_predict = pd.DataFrame(y_predict,index = y_test.index,columns=['Prediction'])

# 그래프
fig, axes = plt.subplots(1, 1, figsize=(12, 4))
plt.plot(y_train, label='Train')        # 훈련 데이터
plt.plot(y_test, label='Test')          # 테스트 데이터
plt.plot(y_predict, label='Prediction')
plt.xlim
# 예측 데이터
plt.legend()
plt.show()

In [ ]:
def forecast_one_step():
    fc, conf_int = model.predict(n_periods=5 # 한 스텝씩!
        , return_conf_int=True)              # 신뢰구간 출력
    return (
        fc.tolist()[0],
        np.asarray(conf_int).tolist()[0]
    )

In [ ]:
forecasts = []
y_pred = []
pred_upper = []
pred_lower = []

for new_ob in y_test:
    fc, conf = forecast_one_step()
    y_pred.append(fc)
    pred_upper.append(conf[1])
    pred_lower.append(conf[0])

    ## 모형 업데이트 !!
    model.update(new_ob)

In [ ]:
pd.DataFrame({"test": y_test, "pred": y_pred})

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = go.Figure([
    # 훈련 데이터-------------------------------------------------------
    go.Scatter(x = y_train.index, y = y_train, name = "Train", mode = 'lines'
              ,line=dict(color = 'royalblue'))
    # 테스트 데이터------------------------------------------------------
    , go.Scatter(x = y_test.index, y = y_test, name = "Test", mode = 'lines'
                ,line = dict(color = 'rgba(0,0,30,0.5)'))
    # 예측값-----------------------------------------------------------
    , go.Scatter(x = y_test.index, y = y_pred, name = "Prediction", mode = 'lines'
                     ,line = dict(color = 'red', dash = 'dot', width=3))
    
    # 신뢰 구간---------------------------------------------------------
    , go.Scatter(x = y_test.index.tolist() + y_test.index[::-1].tolist() 
                ,y = pred_upper + pred_lower[::-1] ## 상위 신뢰 구간 -> 하위 신뢰 구간 역순으로
                ,fill='toself'
                ,fillcolor='rgba(0,0,30,0.1)'
                ,line=dict(color='rgba(0,0,0,0)')
                ,hoverinfo="skip"
                ,showlegend=False)
])

fig.update_layout(height=400, width=1000, title_text="ARIMA(0,1,0)모형")
fig.show()

In [ ]:
def MAPE(y_test, y_pred):
    	return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 
    
print(f"MAPE: {MAPE(y_test, y_pred):.3f}")